### A dirty little trick to avoid having to deal with the notion of relative path. 

In [17]:
import os
os.chdir('/home/juagudelo/HOMEdev/drought_pred_hybrid/')

In [18]:
os.getcwd()

'/home/juagudelo/HOMEdev/drought_pred_hybrid'

## From here, this notebook is going to be used to evaluate the best model issued from the hyperparameter tuning over the test set.

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
from tqdm import tqdm
from scipy.interpolate import interp1d
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import f1_score, mean_absolute_error

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import models
import utilities

In [20]:
# Fixing a seed to warrant the reproducibility
torch.manual_seed(21)
np.random.seed(21)

In [21]:
# Initializing the model
num_categorical_features=7
num_numerical_features=22
num_time_series_features=21
batch_size=128
output_weeks=6
# Hyperparameters
num_epochs_entire=3
hidden_size= 160
num_lstm_layers= 5
embedding_dims= 250
num_fc_tabular_layers= 2
num_fc_combined_layers= 5
dropout= 0.2
learning_rate= 5.414458430824775e-05

In [22]:
# Importing the list of unique categories for the categorical features
with open(f"data/processed_dataFrames/list_cat.pickle", "rb") as f:
    list_cat = pickle.load(f)

# Importing the 

In [23]:
# Setting up the loaders
dfs=utilities.load_dataFrames()
valid_loader=utilities.create_dataLoader(X_static=dfs["X_tabular_valid"],
                                         X_static_cat=dfs["X_tabular_cat_valid"],
                                         X_time=dfs["X_time_valid"],
                                         y_target=dfs["y_target_valid"],
                                         output_weeks=output_weeks,
                                         y_past=None,
                                         batch_size=128,
                                         shuffle=False
                                         )
test_loader=utilities.create_dataLoader(X_static=dfs["X_tabular_test"],
                                        X_static_cat=dfs["X_tabular_cat_test"],
                                        X_time=dfs["X_time_test"],
                                        y_target=dfs["y_target_test"],
                                        output_weeks=output_weeks,
                                        y_past=None,
                                        batch_size=128,
                                        shuffle=False
                                        )

In [24]:
# set up the device
device=torch.device("cpu")
print(f"Using device: {device}")
print(torch.cuda.get_device_name(device=None))

model = models.HybridModel(num_categorical_features,
                           list_cat,
                           num_numerical_features,
                           num_time_series_features,
                           hidden_size,
                           num_lstm_layers,
                           dropout,
                           embedding_dims,
                           num_fc_tabular_layers,
                           num_fc_combined_layers,
                           output_size=output_weeks,
)

model.load_state_dict(torch.load("models/MH_Hyper/MH_Hyper_1.pt"))
model.to(device)

Using device: cpu
NVIDIA T1000 8GB


/tmp/ipykernel_63997/1993821846.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("models/MH_Hyper/MH_Hyper_1.pt"))


HybridModel(
  (embeddings): ModuleList(
    (0-4): 5 x Embedding(7, 250)
    (5): Embedding(6, 250)
    (6): Embedding(8, 250)
  )
  (tabular_fc_layers): Sequential(
    (0): Linear(in_features=1772, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
  )
  (lstm): LSTM(21, 160, num_layers=5, batch_first=True)
  (attention): Linear(in_features=160, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc_after_context): Linear(in_features=160, out_features=64, bias=True)
  (combined_fc_layers): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear

In [25]:
def predict(x, static, static_cat):
    out = model(static_cat, static, x)
    return out

In [26]:
dict_map = {
    "y_pred": [],
    "y_pred_rounded": [],
    # "fips": [],
    # "date": [],
    "y_true": [],
    "week": [],
}
i = 0
for static, static_cat, x, y in tqdm(
    test_loader,
    desc="Test predictions...",
):
    x, static, y = x.to(device), static.to(device), y.to(device)
    with torch.no_grad():
        pred = predict(x, static, static_cat).clone().detach()
    for w in range(output_weeks):
        dict_map["y_pred"] += [float(p[w]) for p in pred]
        dict_map["y_pred_rounded"] += [int(p.round()[w]) for p in pred]
        # dict_map["fips"] += [f[1][0] for f in valid_fips[i : i + len(x)]]
        # dict_map["date"] += [f[1][1] for f in valid_fips[i : i + len(x)]]
        dict_map["y_true"] += [float(item[w]) for item in y]
        dict_map["week"] += [w] * len(x)
    i += len(x)
df = pd.DataFrame(dict_map)

Validation predictions...: 100%|██████████| 69/69 [00:06<00:00, 10.49it/s]


In [29]:
df

,y_pred,y_pred_rounded,y_true,week
0,1.884420,2,2.8891,0
1,0.000700,0,0.8519,0
2,0.000645,0,0.0000,0
3,1.515110,2,1.4617,0
4,1.479904,1,1.6942,0
...,...,...,...,...
52603,1.178136,1,0.0000,5
52604,1.282593,1,2.0000,5
52605,0.006253,0,0.0000,5
52606,0.006797,0,0.0000,5


In [27]:
for w in range(6):
    wdf = df[df['week']==w]
    mae = mean_absolute_error(wdf['y_true'], wdf['y_pred']).round(3)
    f1 = f1_score(wdf['y_true'].round(),wdf['y_pred'].round(), average='macro').round(3)
    print(f"Week {w+1}", f"MAE {mae}", f"F1 {f1}")

Week 1 MAE 0.143 F1 0.677
Week 2 MAE 0.198 F1 0.574
Week 3 MAE 0.246 F1 0.513
Week 4 MAE 0.289 F1 0.471
Week 5 MAE 0.325 F1 0.428
Week 6 MAE 0.357 F1 0.403
